# Artificial and Computational Intelligence Assignment 1

## Problem solving by Uninformed & Informed Search
### By Sneha Banerjee

# Problem Solving Agent (PSA) Definition

## Overview
The Problem Solving Agent (PSA) in this scenario is Rohan, who is designed to solve a path-finding problem on a campus layout. Its objective is to find a valid path from a given start state to a goal state (e.g., from "Admission Office" to "Exit") while visiting every node exactly once and minimizing the total travel cost.

## Agent Components
- **State Representation:**  
  Each state is represented as a tuple:  
  `(current_node, path, visited)`  
  where:  
  - `current_node` is the node currently being explored.  
  - `path` is a list of nodes visited in sequence.  
  - `visited` is a set of nodes that have been visited.

- **Transition Model:**  
  The agent uses a transition model to generate successor states. For a given state, it examines the outgoing edges from the `current_node` in the graph. A successor is generated if the neighbor node has not yet been visited (or if it is the goal, to allow completion).

- **Goal Test:**  
  The goal test function checks if the current state meets the following conditions:  
  1. The `current_node` equals the goal node.  
  2. All nodes in the graph have been visited (i.e., the size of the `visited` set equals the total number of nodes).

- **Search Algorithms:**  
  The PSA incorporates multiple search algorithms to explore the solution space:
  - **Greedy Best First Search (GBFS):** Uses a priority queue guided by heuristic values to quickly find a valid path.
  - **Genetic Algorithm (GA):** Evolves a population of candidate paths to find a solution by exploring diverse solutions and escaping local optima.
  - *(Optionally, Bidirectional Search or other methods can be incorporated.)*

- **Dynamic Input Handling:**  
  The agent accepts dynamic input from the user to define the graph, and to specify the start and goal states.

## Agent Architecture
1. **Input Module:**  
   - Reads user input for the number of nodes, node names, edges (with weights), and the start/goal states.
2. **State Representation Module:**  
   - Constructs and maintains the state in the form `(current_node, path, visited)`.
3. **Transition Model Module:**  
   - Generates successor states based on the current state's outgoing edges.
4. **Search Module:**  
   - Implements the search algorithms (GBFS, Genetic Algorithm) to explore the state space.
5. **Output Module:**  
   - Displays the found solution path, total cost, and performance metrics (runtime, memory usage).
6. **Comparative Analysis Module:**  
   - Provides a comparative analysis of the search algorithms based on dynamic performance measurements.

## Pseudocode of the Agent


```python
function ProblemSolvingAgent():
   graph, start, goal = get_initial_state()  # Dynamic input for the graph and states
   heuristic_values = compute_heuristic_values(graph, goal)
   # Invoke Greedy Best First Search (GBFS)
   gbfs_path, gbfs_cost = greedy_best_first_search(graph, start, goal, heuristic_values)
   # Invoke Genetic Algorithm (GA)
   ga_path, ga_cost, fitness_history = genetic_algorithm(graph, start, goal)
   # Output the results
   display_results(gbfs_path, gbfs_cost, ga_path, ga_cost)
   perform_comparative_analysis()



## Setting Initial State by handing dynamic inputs

In [1]:
# Code Block : Set Initial State (Must handle dynamic inputs)
graph = {}

def set_initial_state():
    """
    Dynamically sets the initial state for the PSA agent.
    This includes:
      - Creating a graph where each node and its outgoing edges (with weights) are provided by the user.
      - Defining the start and goal nodes.
    
    Returns:
      graph: Dictionary representing the graph.
      start: Start node (string).
      goal: Goal node (string).
    """
    #graph = {}

    # Get the number of nodes and their names
    num_nodes = int(input("Enter the number of nodes in the graph: "))
    nodes = []
    for i in range(num_nodes):
        node = input(f"Enter the name for node {i+1}: ").strip()
        nodes.append(node)
        graph[node] = {}  # Initialize empty edge dictionary for this node

    print("\nNow, enter the edges in the format: <source> <target> <weight>")
    print("Type 'done' when you have finished adding edges.")
    while True:
        edge_input = input("Edge: ").strip()
        if edge_input.lower() == "done":
            break
        try:
            source, target, weight = edge_input.split()
            weight = float(weight)
            if source not in graph:
                print(f"Source node '{source}' not recognized. Please enter a valid source node.")
                continue
            if target not in graph:
                print(f"Target node '{target}' not recognized. Please enter a valid target node.")
                continue
            graph[source][target] = weight
        except ValueError:
            print("Invalid input format. Please use: <source> <target> <weight>")
    
    # Set the start and goal nodes
    start = input("\nEnter the start node: ").strip()
    goal = input("Enter the goal node: ").strip()

    # Validate start and goal
    if start not in graph:
        raise ValueError(f"Start node '{start}' is not in the graph.")
    if goal not in graph:
        raise ValueError(f"Goal node '{goal}' is not in the graph.")

    return graph, start, goal

# Example usage:
if __name__ == "__main__":
    graph, start, goal = set_initial_state()
    print("\nInitial Graph:")
    for node, edges in graph.items():
        print(f"{node} -> {edges}")
    print(f"\nStart Node: {start}")
    print(f"Goal Node: {goal}")



Now, enter the edges in the format: <source> <target> <weight>
Type 'done' when you have finished adding edges.

Initial Graph:
Admission_Office -> {'Hostel_Office': 2.0, 'Library': 4.0}
Hostel_Office -> {'Admission_Office': 2.0, 'Library': 4.0, 'Canteen': 6.0, 'Hostel_visit': 2.0}
Library -> {'Admission_Office': 4.0, 'Hostel_Office': 4.0, 'Dept_visit': 3.0, 'Canteen': 7.0}
Dept_visit -> {'Library': 3.0, 'Canteen': 2.0, 'Exit': 5.0}
Canteen -> {'Library': 7.0, 'Dept_visit': 2.0, 'Hostel_Office': 6.0, 'Hostel_visit': 6.0, 'Exit': 8.0}
Hostel_visit -> {'Canteen': 6.0, 'Hostel_Office': 2.0, 'Exit': 4.0}
Exit -> {'Hostel_visit': 4.0, 'Canteen': 8.0, 'Dept_visit': 5.0}

Start Node: Admission_Office
Goal Node: Exit


## Setting the matrix for transition & cost

In [2]:
#Code Block : Set the matrix for transition & cost (as relevant for the given problem)
# Code Block : Set the matrix for transition & cost (as relevant for the given problem)

def set_transition_cost_matrices(graph):
    """
    Given a graph as a dictionary of dictionaries, this function creates:
      - A transition matrix, where each cell [i][j] is 1 if there is an edge from node i to node j, else 0.
      - A cost matrix, where each cell [i][j] is the cost of the edge from node i to node j if it exists;
        otherwise, float('inf') if i != j and 0 if i == j.
    
    Matrices are represented as nested dictionaries with keys as node names.
    """
    nodes = list(graph.keys())
    # Initialize matrices: transition matrix with 0's and cost matrix with infinity.
    transition_matrix = {node: {other: 0 for other in nodes} for node in nodes}
    cost_matrix = {node: {other: float('inf') for other in nodes} for node in nodes}
    
    # Set diagonal values: no cost to remain at the same node.
    for node in nodes:
        cost_matrix[node][node] = 0
    
    # Populate matrices based on the graph's edges.
    for source in graph:
        for target, cost in graph[source].items():
            transition_matrix[source][target] = 1  # There is a valid transition from source to target.
            cost_matrix[source][target] = cost       # Set the cost for that transition.
    
    return transition_matrix, cost_matrix

# Example usage:
if __name__ == "__main__":
    # Example graph (this can also be obtained dynamically as in the initial state block)

    transition_matrix, cost_matrix = set_transition_cost_matrices(graph)
    
    print("Transition Matrix:")
    for source in transition_matrix:
        print(f"{source}: {transition_matrix[source]}")
    
    print("\nCost Matrix:")
    for source in cost_matrix:
        print(f"{source}: {cost_matrix[source]}")


Transition Matrix:
Admission_Office: {'Admission_Office': 0, 'Hostel_Office': 1, 'Library': 1, 'Dept_visit': 0, 'Canteen': 0, 'Hostel_visit': 0, 'Exit': 0}
Hostel_Office: {'Admission_Office': 1, 'Hostel_Office': 0, 'Library': 1, 'Dept_visit': 0, 'Canteen': 1, 'Hostel_visit': 1, 'Exit': 0}
Library: {'Admission_Office': 1, 'Hostel_Office': 1, 'Library': 0, 'Dept_visit': 1, 'Canteen': 1, 'Hostel_visit': 0, 'Exit': 0}
Dept_visit: {'Admission_Office': 0, 'Hostel_Office': 0, 'Library': 1, 'Dept_visit': 0, 'Canteen': 1, 'Hostel_visit': 0, 'Exit': 1}
Canteen: {'Admission_Office': 0, 'Hostel_Office': 1, 'Library': 1, 'Dept_visit': 1, 'Canteen': 0, 'Hostel_visit': 1, 'Exit': 1}
Hostel_visit: {'Admission_Office': 0, 'Hostel_Office': 1, 'Library': 0, 'Dept_visit': 0, 'Canteen': 1, 'Hostel_visit': 0, 'Exit': 1}
Exit: {'Admission_Office': 0, 'Hostel_Office': 0, 'Library': 0, 'Dept_visit': 1, 'Canteen': 1, 'Hostel_visit': 1, 'Exit': 0}

Cost Matrix:
Admission_Office: {'Admission_Office': 0, 'Hostel_O

## Defining the function to design the Transition Model/Successor function. These functions would be called when search algorithms are implemented


### Get dynamic user input for state and goal, then generate successors using the Transition Model

In [17]:
import shlex

def get_state_and_goal():
    """
    Prompts the user to enter the current state and the goal state.
    
    Returns:
      tuple: (current_state, goal_state)
    """
    current_state = input("Enter the current state: ").strip()
    goal_state = input("Enter the goal state: ").strip()
    return current_state, goal_state

def transition_model(state, graph, goal):
    """
    Given the current state, returns a list of valid successor states based on the graph.
    
    Each state is represented as a tuple:
      (current_node, path, visited)
      - current_node: The current location (string)
      - path: A list of nodes representing the sequence visited so far
      - visited: A set of nodes that have been visited
    
    Parameters:
      state : tuple
          The current state as (current_node, path, visited).
      graph : dict
          The graph represented as a dictionary of dictionaries, where each key is a node and 
          its value is another dictionary mapping neighboring nodes to edge costs.
      goal : str
          The goal node.
    
    Returns:
      successors : list
          A list of successor states, each in the form (next_node, new_path, new_visited).
    """
    current_node, path, visited = state
    successors = []
    
    
    for neighbor, cost in graph[current_node].items():
        # Only add the neighbor if it hasn't been visited yet.
        if neighbor not in visited:
            new_state = (neighbor, path + [neighbor], visited | {neighbor})
            successors.append(new_state)
    
    return successors

# Example usage when state and goal are provided by user input.
if __name__ == "__main__":
    # Define the graph as specified.
    
    # Get dynamic input for the current state and goal.
    current_state, goal_state = get_state_and_goal()
    print("\nSelected Current State:", current_state)
    print("Selected Goal State:", goal_state)
    
    # Construct the initial state representation for the search:
    # (current_node, path, visited)
    initial_state = (current_state, [current_state], {current_state})
    
    # Generate and print the successor states using the transition model.
    successors = transition_model(initial_state, graph, goal_state)
    print("\nSuccessor States for state '{}' with goal '{}':".format(current_state, goal_state))
    for succ in successors:
        print(succ)




Selected Current State: Admission_Office
Selected Goal State: Exit

Successor States for state 'Admission_Office' with goal 'Exit':
('Hostel_Office', ['Admission_Office', 'Hostel_Office'], {'Hostel_Office', 'Admission_Office'})
('Library', ['Admission_Office', 'Library'], {'Library', 'Admission_Office'})


## DYNAMIC INPUT

## Function & call to get inputs (start/end state)

In [26]:
def get_start_goal_states(graph):
    """
    Display the possible states to choose from and get user input for the start and goal states.
    
    Parameters:
      graph : dict
          The graph represented as a dictionary where keys are the available states.
    
    Returns:
      tuple: (start, goal)
          The start and goal states as entered by the user.
    """
    # Display the available states
    print("Available states:")
    for state in graph.keys():
        print(" -", state)
    
    # Get dynamic input for the start state
    start = input("Enter the start state from the available states: ").strip()
    while start not in graph:
        print("Invalid start state. Please choose from the available states.")
        start = input("Enter the start state from the available states: ").strip()
        
    # Get dynamic input for the goal state
    goal = input("Enter the goal state from the available states: ").strip()
    while goal not in graph:
        print("Invalid goal state. Please choose from the available states.")
        goal = input("Enter the goal state from the available states: ").strip()
    
    return start, goal

if __name__ == "__main__":
    
    # Get the start and goal states dynamically from the user
    start, goal = get_start_goal_states(graph)
    print("\nSelected Start State:", start)
    print("Selected Goal State:", goal)


Available states:
 - Admission_Office
 - Hostel_Office
 - Library
 - Dept_visit
 - Canteen
 - Hostel_visit
 - Exit

Selected Start State: Admission_Office
Selected Goal State: Exit


## Function to handle goal test using dynamic inputs. This would be called while search algorithms are implemented

In [ ]:
def goal_test(state, goal, graph):
    """
    Checks whether the current state meets the goal criteria.
    
    A state is represented as a tuple:
        (current_node, path, visited)
    where:
        - current_node: the node currently being explored (string)
        - path: a list of nodes representing the sequence visited so far
        - visited: a set of nodes that have been visited

    The goal is achieved if:
      1. The current node is the goal node.
      2. All nodes in the graph have been visited (dynamically determined from the graph).

    Parameters:
      state : tuple
          The current state in the form (current_node, path, visited).
      goal : str
          The goal node.
      graph : dict
          The graph represented as a dictionary of dictionaries, where keys are node names.
    
    Returns:
      bool: True if the state meets the goal criteria, False otherwise.
    """
    current_node, path, visited = state
    # Uncomment the following line if you want to stop only when all nodes are visited:
    return current_node == goal and len(visited) == len(graph)
    #return current_node == goal


if __name__ == "__main__":
    # Example graph for demonstration:
    
    current_node = input("Enter the current state: ").strip()
    goal = input("Enter the goal state: ").strip()
    
    # Construct the state as (current_node, path, visited)
    state = (current_node, [current_node], {current_node})
    
    result = goal_test(state, goal, graph)
    print(f"Current Node: {current_node}, Goal: {goal}")
    print("Goal Test Result:", result)




Current Node: Admission_Office, Goal: Exit
Goal Test Result: False


# Definition of Greedy Best First Search Algorithm

## Greedy Best First Search (GBFS) Algorithm

## Overview
Greedy Best First Search (GBFS) is a heuristic-driven search algorithm that expands the node that appears to be closest to the goal, based solely on an evaluation function (heuristic). It uses a priority queue to select the next node to explore according to its estimated cost to reach the goal. While GBFS can often find a solution quickly, it is not guaranteed to find the optimal (shortest) path.

## Algorithm Details
- **Input:**  
  - A graph (or search space)
  - A start node
  - A goal node
  - A heuristic function `h(n)` that estimates the cost from node `n` to the goal.
- **Output:**  
  A path from the start node to the goal node, if one exists.

## Pseudocode

```pseudo
function GBFS(graph, start, goal, h):
    // Initialize the open list (priority queue) with the start node and its heuristic value.
    open_list = PriorityQueue()
    open_list.push(start, h(start))
    
    // Initialize the closed set to keep track of visited nodes.
    closed_set = {}

    // Optionally, maintain a mapping for reconstructing the path.
    parent = {}

    while open_list is not empty:
        // Remove the node with the lowest heuristic value.
        current = open_list.pop()

        // If the goal is reached, reconstruct and return the path.
        if current == goal:
            return reconstruct_path(parent, goal)
        
        // Mark the current node as visited.
        closed_set.add(current)
        
        // Process each neighbor of the current node.
        for each neighbor in graph.neighbors(current):
            if neighbor is not in closed_set:
                parent[neighbor] = current
                open_list.push(neighbor, h(neighbor))
    
    return "No path found"


## Function for GBFS implementation using the given graph which was earlier provided as input by the user

In [29]:
import heapq

def compute_cost(path, graph):
    """
    Compute the total cost of traversing the given path using the graph.
    The path is a list of nodes.
    """
    total_cost = 0
    for i in range(len(path) - 1):
        source = path[i]
        target = path[i + 1]
        total_cost += graph[source].get(target, float('inf'))
    return total_cost


def greedy_best_first_search(graph, start, goal, heuristic_values):
    """
    Implements Greedy Best First Search (GBFS) to find a valid path from start to goal.
    The goal is defined as reaching 'Exit' after having visited all nodes.
    
    A modified heuristic is used that rewards states with more visited nodes.
    """
    def modified_heuristic(state):
        current_node, path, visited = state
        base = heuristic_values[current_node]
        total_nodes = len(graph)
        bonus = (len(visited) / total_nodes) * 100  # Adjust bonus scaling as needed.
        return base - bonus

    # Initial state: (current_node, path, visited_set)
    initial_state = (start, [start], {start})
    
    # Priority queue stores tuples: (modified_heuristic, state)
    open_list = []
    heapq.heappush(open_list, (modified_heuristic(initial_state), initial_state))
    
    while open_list:
        current_f, state = heapq.heappop(open_list)
        current_node, path, visited = state
        
        # Debug (optional): print current state info
        # print("Expanding:", current_node, "Visited:", visited)
        
        if goal_test(state, goal, graph):
            total_cost = compute_cost(path, graph)
            return path, total_cost
        
        for succ in transition_model(state, graph, goal):
            heapq.heappush(open_list, (modified_heuristic(succ), succ))
    
    return ["Error: Path Not Found"], float('inf')


# Define heuristic values for each node.
# For nodes with a direct edge to "Exit", use that cost; otherwise, assign a high value.
heuristic_values = {}
for node in graph:
    if node == "Exit":
        heuristic_values[node] = 0
    else:
        heuristic_values[node] = graph[node].get("Exit", 1000)

# Set start and goal nodes.
start, goal = get_start_goal_states(graph)


if __name__ == "__main__":
    path, cost = greedy_best_first_search(graph, start, goal, heuristic_values)
    print("GBFS Path:", path)
    print("GBFS Total Cost:", cost)


Available states:
 - Admission_Office
 - Hostel_Office
 - Library
 - Dept_visit
 - Canteen
 - Hostel_visit
 - Exit
GBFS Path: ['Admission_Office', 'Hostel_Office', 'Hostel_visit', 'Canteen', 'Library', 'Dept_visit', 'Exit']
GBFS Total Cost: 25.0


##	Definition of Genetic Algorithm

# Genetic Algorithm (GA)

## Overview
A Genetic Algorithm is a search heuristic inspired by the process of natural selection. It is used for solving optimization and search problems by evolving a population of candidate solutions over several generations using bio-inspired operators such as selection, crossover, and mutation.

## Algorithm Details
- **Input:**  
  - A population of candidate solutions (individuals), typically represented as strings, arrays, or permutations.
  - A fitness function to evaluate each candidate's quality.
  - Parameters such as population size, number of generations, crossover probability, and mutation probability.
- **Output:**  
  - A candidate solution that approximates the optimal solution to the problem.

## Pseudocode

```pseudo
function GeneticAlgorithm(population_size, generations, mutation_rate):
    // Initialize a random population of candidate solutions.
    population = generate_random_population(population_size)
    
    for generation from 1 to generations:
        // Evaluate the fitness of each individual in the population.
        fitness_scores = evaluate_fitness(population)
        
        // Select individuals for reproduction based on their fitness.
        selected = selection(population, fitness_scores)
        
        // Create a new population using crossover and mutation.
        new_population = []
        while size(new_population) < population_size:
            parent1, parent2 = select_two(selected)
            child = crossover(parent1, parent2)
            child = mutate(child, mutation_rate)
            new_population.append(child)
        
        population = new_population
    
    // Return the best individual from the final population.
    best_individual = select_best(population)
    return best_individual


## Function for Genetic Algorithm implementation

In [30]:
import random

def genetic_algorithm(graph, start, goal, population_size=100, generations=500, mutation_rate=0.1):
    """
    Implements a Genetic Algorithm to find a valid path that visits every intermediate node exactly once,
    starting at 'start' and ending at 'goal'. The intermediate nodes are permuted.
    
    Parameters:
      graph (dict): The graph represented as an adjacency list (dictionary of dictionaries).
      start (str): The starting node.
      goal (str): The goal node.
      population_size (int): Number of individuals in the population.
      generations (int): Number of generations to evolve.
      mutation_rate (float): Probability of mutation for an individual.
      
    Returns:
      tuple: (best_path, best_distance, best_fitness_history)
             best_path is a list representing the sequence of nodes from start to goal.
             best_distance is the total cost of that path.
             best_fitness_history is a list recording the best fitness value (negative cost) per generation.
             If no valid path is found, returns (["Error: Path Not Found"], float('inf'), best_fitness_history).
    """
    # Extract intermediate nodes: all nodes except start and goal.
    nodes = list(graph.keys())
    nodes.remove(start)
    nodes.remove(goal)
    
    def fitness_function(path):
        """
        Compute the fitness of an individual (a permutation of intermediate nodes).
        The total cost is computed as:
          cost from start to first node +
          cost along consecutive nodes in the path +
          cost from last node to goal.
        If the path does not cover all intermediate nodes exactly once, return infinity as a penalty.
        
        Fitness is defined as the negative total cost, so a lower cost gives a higher fitness.
        """
        try:
            total_cost = graph[start][path[0]]  # cost from start to first intermediate
            for i in range(len(path) - 1):
                total_cost += graph[path[i]][path[i+1]]
            total_cost += graph[path[-1]][goal]   # cost from last intermediate to goal
            # Ensure all intermediate nodes are used exactly once.
            if len(set(path)) != len(nodes):
                return float('inf')
            return -total_cost
        except KeyError:
            return float('inf')
    
    def create_individual():
        """Create a random individual by shuffling the intermediate nodes."""
        individual = nodes[:]
        random.shuffle(individual)
        return individual
    
    def mutate(individual):
        """Mutate an individual by swapping two randomly chosen nodes with probability mutation_rate."""
        if random.random() < mutation_rate:
            i, j = random.sample(range(len(individual)), 2)
            individual[i], individual[j] = individual[j], individual[i]
    
    def crossover(parent1, parent2):
        """
        Perform crossover between two parent individuals using one-point crossover.
        The child is built by taking the first part from parent1 and then filling in the remaining nodes
        in the order they appear in parent2, avoiding duplicates.
        """
        cut = random.randint(1, len(parent1) - 1)
        child = parent1[:cut]
        child += [node for node in parent2 if node not in child]
        return child
    
    # Initialize the population with random individuals.
    population = [create_individual() for _ in range(population_size)]
    best_fitness_history = []
    
    for generation in range(generations):
        # Sort population by fitness (lower total cost -> higher fitness, since fitness = -total_cost).
        population.sort(key=lambda ind: fitness_function(ind))
        best_fitness = fitness_function(population[0])
        best_fitness_history.append(best_fitness)
        
        # Elitism: retain the top 10% of individuals.
        elite_count = max(1, population_size // 10)
        next_generation = population[:elite_count]
        
        # Generate new individuals until the population is replenished.
        while len(next_generation) < population_size:
            parent1, parent2 = random.sample(population[:max(2, population_size // 2)], 2)
            child = crossover(parent1, parent2)
            mutate(child)
            next_generation.append(child)
        population = next_generation
    
    best_individual = min(population, key=lambda ind: fitness_function(ind))
    if fitness_function(best_individual) == float('inf'):
        return ["Error: Path Not Found"], float('inf'), best_fitness_history
    
    best_path = [start] + best_individual + [goal]
    best_distance = -fitness_function(best_individual)
    return best_path, best_distance, best_fitness_history


start, goal = get_start_goal_states(graph)

# --- Main Execution Example ---
if __name__ == "__main__":
    best_path, best_distance, best_fitness_history = genetic_algorithm(graph, start, goal, population_size=100, generations=500, mutation_rate=0.1)
    print("Genetic Algorithm Best Path:", best_path)
    print("Genetic Algorithm Best Total Cost:", best_distance)


Available states:
 - Admission_Office
 - Hostel_Office
 - Library
 - Dept_visit
 - Canteen
 - Hostel_visit
 - Exit


Genetic Algorithm Best Path: ['Admission_Office', 'Hostel_Office', 'Hostel_visit', 'Canteen', 'Library', 'Dept_visit', 'Exit']
Genetic Algorithm Best Total Cost: 25.0


## Function for Bidirectional Search Algorithm implementation

In [ ]:
import heapq

def compute_cost(path, graph):
    """
    Compute the total cost of traversing the given path using the graph.
    The path is a list of nodes.
    """
    total_cost = 0
    for i in range(len(path) - 1):
        source = path[i]
        target = path[i + 1]
        total_cost += graph[source].get(target, float('inf'))
    return total_cost

def build_reverse_graph(graph):
    """
    Build and return the reverse of the given directed graph.
    For each edge u -> v with cost, the reverse graph has an edge v -> u with the same cost.
    """
    reverse_graph = {node: {} for node in graph}
    for u in graph:
        for v, cost in graph[u].items():
            reverse_graph[v][u] = cost
    return reverse_graph

def bidirectional_search(graph, start, goal):
    """
    Performs Bidirectional Search to find a Hamiltonian path from 'start' to 'goal'
    (i.e., a path that visits every node exactly once, with the final node equal to goal).

    Returns:
        (path, total_cost): Tuple containing the complete path and its total cost,
                            or (["Error: Path Not Found"], inf) if no valid path is found.
    """
    # Build reverse graph for backward expansion.
    reverse_graph = build_reverse_graph(graph)
    
    # Each state is represented as (node, path, visited)
    # Initialize forward frontier from start and backward frontier from goal.
    forward_frontier = [(start, [start], {start})]
    backward_frontier = [(goal, [goal], {goal})]
    
    # Maintain dictionaries for visited states (mapping node to state) in each direction.
    forward_visited = {start: (start, [start], {start})}
    backward_visited = {goal: (goal, [goal], {goal})}
    
    while forward_frontier and backward_frontier:
        # ----- Expand Forward Frontier -----
        new_forward_frontier = []
        for state in forward_frontier:
            current_node, path, visited = state
            for neighbor, cost in graph[current_node].items():
                if neighbor not in visited:
                    new_state = (neighbor, path + [neighbor], visited | {neighbor})
                    if neighbor not in forward_visited:
                        forward_visited[neighbor] = new_state
                        new_forward_frontier.append(new_state)
        forward_frontier = new_forward_frontier

        # Check for intersection between forward and backward frontiers.
        intersect = set(forward_visited.keys()).intersection(set(backward_visited.keys()))
        if intersect:
            meet = intersect.pop()
            f_state = forward_visited[meet]  # (node, f_path, f_visited)
            b_state = backward_visited[meet]  # (node, b_path, b_visited)
            # Merge paths. b_state[1] is the path from goal to meet, so reverse it (excluding the meeting node)
            combined_path = f_state[1] + b_state[1][-2::-1]
            combined_visited = f_state[2] | b_state[2]
            if len(combined_visited) == len(graph):
                return combined_path, compute_cost(combined_path, graph)
        
        # ----- Expand Backward Frontier -----
        new_backward_frontier = []
        for state in backward_frontier:
            current_node, path, visited = state
            for neighbor, cost in reverse_graph[current_node].items():
                if neighbor not in visited:
                    new_state = (neighbor, [neighbor] + path, visited | {neighbor})
                    if neighbor not in backward_visited:
                        backward_visited[neighbor] = new_state
                        new_backward_frontier.append(new_state)
        backward_frontier = new_backward_frontier

        # Check for intersection again.
        intersect = set(forward_visited.keys()).intersection(set(backward_visited.keys()))
        if intersect:
            meet = intersect.pop()
            f_state = forward_visited[meet]
            b_state = backward_visited[meet]
            combined_path = f_state[1] + b_state[1][-2::-1]
            combined_visited = f_state[2] | b_state[2]
            if len(combined_visited) == len(graph):
                return combined_path, compute_cost(combined_path, graph)
    
    return ["Error: Path Not Found"], float('inf')

# -------------------------------
# Example Usage
# -------------------------------
if __name__ == "__main__":
 
    
    start, goal = get_state_and_goal() 
    
    path, cost = bidirectional_search(graph, start, goal)
    print("Bidirectional Search Path:", path)
    print("Bidirectional Search Total Cost:", cost)


Bidirectional Search Path: ['Error: Path Not Found']
Bidirectional Search Total Cost: inf


## 	Calling the search algorithms - GBFS, Genetic Algorithm and Bidirectional Search

In [35]:
import time

def main():
    
    # Get dynamic input for the start and goal states.
    start, goal = get_start_goal_states(graph)
    print("\nSelected Start State:", start)
    print("Selected Goal State:", goal)
    
    # Precompute heuristic values for GBFS.
    # For each node, if there's a direct edge to the goal, we use its cost; otherwise, a high value is assigned.
    heuristic_values = {}
    for node in graph:
        if node == goal:
            heuristic_values[node] = 0
        else:
            heuristic_values[node] = graph[node].get(goal, 1000)  # 1000 is an arbitrarily high cost

    # -------------------------------
    # Call Greedy Best First Search (GBFS)
    # -------------------------------
    print("\nCalling Greedy Best First Search (GBFS)...")
    start_time = time.time()
    gbfs_path, gbfs_cost = greedy_best_first_search(graph, start, goal, heuristic_values)
    gbfs_runtime = time.time() - start_time
    print("GBFS Path:", gbfs_path)
    print("GBFS Total Cost:", gbfs_cost)
    print("GBFS Runtime: {:.6f} seconds".format(gbfs_runtime))
    
    # -------------------------------
    # Call Genetic Algorithm (GA)
    # -------------------------------
    print("\nCalling Genetic Algorithm...")
    start_time = time.time()
    ga_path, ga_cost, ga_fitness_history = genetic_algorithm(graph, start, goal,
                                                             population_size=100,
                                                             generations=500,
                                                             mutation_rate=0.1)
    ga_runtime = time.time() - start_time
    print("Genetic Algorithm Best Path:", ga_path)
    print("Genetic Algorithm Total Cost:", ga_cost)
    print("Genetic Algorithm Runtime: {:.6f} seconds".format(ga_runtime))
    
    # -------------------------------
    # Placeholder for Bidirectional Search (if implemented)
    # -------------------------------
    
    print("\nCalling Bidirectional Search...")
    start_time = time.time()
    bd_path, bd_cost = bidirectional_search(graph, start, goal)  # Function to be implemented
    bd_runtime = time.time() - start_time
    print("Bidirectional Search Path:", bd_path)
    print("Bidirectional Search Total Cost:", bd_cost)
    print("Bidirectional Search Runtime: {:.6f} seconds".format(bd_runtime))
    

# Call main if this module is run directly.
if __name__ == "__main__":
    main()


Available states:
 - Admission_Office
 - Hostel_Office
 - Library
 - Dept_visit
 - Canteen
 - Hostel_visit
 - Exit

Selected Start State: Admission_Office
Selected Goal State: Exit

Calling Greedy Best First Search (GBFS)...
GBFS Path: ['Admission_Office', 'Hostel_Office', 'Hostel_visit', 'Canteen', 'Library', 'Dept_visit', 'Exit']
GBFS Total Cost: 25.0
GBFS Runtime: 0.000000 seconds

Calling Genetic Algorithm...
Genetic Algorithm Best Path: ['Admission_Office', 'Hostel_Office', 'Hostel_visit', 'Canteen', 'Library', 'Dept_visit', 'Exit']
Genetic Algorithm Total Cost: 25.0
Genetic Algorithm Runtime: 0.427579 seconds

Calling Bidirectional Search...
Bidirectional Search Path: ['Error: Path Not Found']
Bidirectional Search Total Cost: inf
Bidirectional Search Runtime: 0.015637 seconds


## Comparative Analysis (Time and Space Complexity)
### Dynamically compute time and space usage for both GBFS and Genetic Algorithm

In [34]:
import time
import tracemalloc
import heapq
import random

# # ----------------------------------------------
# # Measurement Functions using tracemalloc and time
# # ----------------------------------------------

def measure_gbfs(graph, start, goal, heuristic_values):
    print("Measuring GBFS...")
    tracemalloc.start()
    t0 = time.time()
    path, cost = greedy_best_first_search(graph, start, goal, heuristic_values)
    runtime = time.time() - t0
    current_mem, peak_mem = tracemalloc.get_traced_memory()
    tracemalloc.stop()
    print("GBFS Path:", path)
    print("GBFS Total Cost:", cost)
    print("GBFS Runtime: {:.6f} seconds".format(runtime))
    print("GBFS Peak Memory Usage: {:.2f} KB".format(peak_mem / 1024))
    return path, cost, runtime, peak_mem

def measure_ga(graph, start, goal, population_size=100, generations=500, mutation_rate=0.1):
    print("Measuring Genetic Algorithm...")
    tracemalloc.start()
    t0 = time.time()
    path, cost, fitness_history = genetic_algorithm(graph, start, goal, population_size, generations, mutation_rate)
    runtime = time.time() - t0
    current_mem, peak_mem = tracemalloc.get_traced_memory()
    tracemalloc.stop()
    print("Genetic Algorithm Path:", path)
    print("Genetic Algorithm Total Cost:", cost)
    print("Genetic Algorithm Runtime: {:.6f} seconds".format(runtime))
    print("Genetic Algorithm Peak Memory Usage: {:.2f} KB".format(peak_mem / 1024))
    return path, cost, runtime, peak_mem

# ----------------------------------------------
# Main Execution: Measure and Print Complexities
# ----------------------------------------------

if __name__ == "__main__":
    print("=== Measuring GBFS ===")
    measure_gbfs(graph, start, goal, heuristic_values)
    
    print("\n=== Measuring Genetic Algorithm ===")
    measure_ga(graph, start, goal, population_size=100, generations=500, mutation_rate=0.1)


=== Measuring GBFS ===
Measuring GBFS...
GBFS Path: ['Admission_Office', 'Hostel_Office', 'Hostel_visit', 'Canteen', 'Library', 'Dept_visit', 'Exit']
GBFS Total Cost: 25.0
GBFS Runtime: 0.000000 seconds
GBFS Peak Memory Usage: 8.30 KB

=== Measuring Genetic Algorithm ===
Measuring Genetic Algorithm...
Genetic Algorithm Path: ['Admission_Office', 'Hostel_Office', 'Hostel_visit', 'Canteen', 'Library', 'Dept_visit', 'Exit']
Genetic Algorithm Total Cost: 25.0
Genetic Algorithm Runtime: 1.606379 seconds
Genetic Algorithm Peak Memory Usage: 188.55 KB




## Greedy Best First Search (GBFS)
- **Time Complexity:**
  - **Worst-case:** O(V log V)  
    (Assuming each node is inserted into the priority queue once, and each insertion/deletion takes O(log V), where V is the number of nodes.)
  - **General Search:** In many search problems, GBFS has exponential worst-case behavior, O(b^d), where _b_ is the branching factor and _d_ is the solution depth. However, in our problem (a well-defined graph with a unique visit constraint), the search is limited to visiting each node exactly once.
- **Space Complexity:**  
  - O(V) due to storage for the priority queue and the visited set.

## Genetic Algorithm (GA)
- **Time Complexity:**
  - O(G * P * E)  
    where:
    - _G_ is the number of generations,
    - _P_ is the population size, and
    - _E_ is the evaluation cost for each individual.  
    (In our case, evaluating a candidate path involves computing the cost along a path, which is proportional to the number of nodes in the path.)
- **Space Complexity:**  
  - O(P * L)  
    where _L_ is the length of an individual (typically the number of intermediate nodes to be permuted), plus overhead for storing fitness values and historical data.

## Bidirectional Search (If Implemented)
- **Time Complexity:**
  - O(b^(d/2))  
    (Under ideal conditions, bidirectional search reduces the effective search depth by half, where _b_ is the branching factor and _d_ is the solution depth.)
- **Space Complexity:**  
  - O(b^(d/2))  
    (Since two frontiers are maintained - one from the start and one from the goal.)

## Comparative Discussion
- **GBFS** is advantageous when a good heuristic is available, as it can efficiently guide the search. However, if the heuristic is not well-informed, it may expand many nodes or even get misled.
- **Genetic Algorithm** offers a robust approach for exploring a large solution space and can escape local minima by evolving a population of solutions. The trade-off is that it may require more computational time and careful parameter tuning (population size, number of generations, mutation rate).
- **Bidirectional Search** (if implemented) can significantly reduce search time by exploring from both the start and goal simultaneously, but it often requires more memory and can be more complex to implement correctly. The algorithm checks for an intersection between the forward and backward frontiers and immediately attempts to merge the paths when a common node is found. However, if the intersection happens early, for example, at a node that is reached before all nodes have been visited in either search, the merged visited set will have an incomplete visited set and, therefore, will not form a valid complete solution.
